<a href="https://colab.research.google.com/github/fadingreflection/new_rag_project/blob/master/LoRA_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth
!pip install datasets accelerate bitsandbytes peft trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.3/351.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: torchao
    Found existing installation: torchao 0.10.0
    Uninstalling torchao-0.10.0:
      

In [2]:
import json
from google.colab import files

# Загружаем файл
uploaded = files.upload()

# Читаем JSON с русскими символами
for filename in uploaded.keys():
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)

# сразу откладываем тестовую выборку
import random
random.seed(42)
random.shuffle(data)
test_data = data[0:150]
train_data=data[150::]

Saving real_data.json to real_data (1).json


In [3]:
import json
import random


# Синтетические данные для балансировки
synthetic_scenarios = []

# 1. ЭРОТИЧЕСКИЙ КОНТЕНТ (добавляем 60 примеров)
erotic_scenarios = [
    # Mild (20 примеров)
    {"text": "Они нежно поцеловались на прощание у лифта.", "level": "Mild", "trigger": "романтический поцелуй"},
    {"text": "Она обняла его и прижалась к груди.", "level": "Mild", "trigger": "нежные объятия"},
    {"text": "Они держались за руки, гуляя по парку.", "level": "Mild", "trigger": "держатся за руки"},
    {"text": "Он нежно поцеловал ее в щеку перед уходом.", "level": "Mild", "trigger": "поцелуй в щеку"},
    {"text": "Они сидели близко, их плечи соприкасались.", "level": "Mild", "trigger": "физическая близость"},


    # Moderate (20 примеров)
    {"text": "Она медленно расстегнула блузку, глядя ему в глаза.", "level": "Moderate", "trigger": "расстегивание одежды"},
    {"text": "Они страстно целовались, его руки скользили по ее спине.", "level": "Moderate", "trigger": "страстный поцелуй с тактильным контактом"},
    {"text": "Он снял с нее платье, не отрывая губ от ее шеи.", "level": "Moderate", "trigger": "раздевание партнера"},
    {"text": "Они лежали в постели, их тела переплелись в поцелуе.", "level": "Moderate", "trigger": "интимная поза в постели"},
    {"text": "Его руки скользили под ее юбкой, она тихо стонала.", "level": "Moderate", "trigger": "тактуальный контакт интимного характера"},

    # Severe (20 примеров)
    {"text": "Он вошел в нее, она громко застонала от удовольствия.", "level": "Severe", "trigger": "явное описание сексуального акта"},
    {"text": "Она опустилась перед ним на колени, расстегивая его брюки.", "level": "Severe", "trigger": "оральный секс"},
    {"text": "Их тела слились в ритмичных движениях, кровать скрипела.", "level": "Severe", "trigger": "детализированное описание полового акта"},
    {"text": "Он ласкал ее грудь, она активно двигалась под ним.", "level": "Severe", "trigger": "детали интимного контакта"},
    {"text": "Она сидела сверху, ее грудь покачивалась в такт движениям.", "level": "Severe", "trigger": "порнографическое описание"},


]

# 2. НАСИЛИЕ (добавляем 40 примеров)
violence_scenarios = [
    # Mild (15 примеров)
    {"text": "Они громко спорили, угрожая друг другу.", "level": "Mild", "trigger": "вербальная угроза"},
    {"text": "Он схватил ее за руку, требуя объяснений.", "level": "Mild", "trigger": "физическое удержание"},
    {"text": "Она оттолкнула его в порыве гнева.", "level": "Mild", "trigger": "толчок без последствий"},
    {"text": "Они с силой столкнулись плечами при встрече.", "level": "Mild", "trigger": "агрессивный контакт"},

    # Moderate (15 примеров)
    {"text": "Он ударил кулаком по столу, оставив вмятину.", "level": "Moderate", "trigger": "разрушение property"},
    {"text": "Из носа пошла кровь после удара.", "level": "Moderate", "trigger": "кровотечение"},
    {"text": "Она упала, ударившись головой о пол.", "level": "Moderate", "trigger": "падение с травмой"},
    {"text": "На его лице остался синяк после драки.", "level": "Moderate", "trigger": "видимые последствия насилия"},

    # Severe (10 примеров)
    {"text": "Он избивал его ногами, пока тот не потерял сознание.", "level": "Severe", "trigger": "жестокое избиение"},
    {"text": "Нож вошел в живот с характерным хлюпающим звуком.", "level": "Severe", "trigger": "применение холодного оружия"},
    {"text": "Она истекала кровью из пулевого ранения в грудь.", "level": "Severe", "trigger": "огнестрельное ранение"},
    {"text": "Они пытали его раскаленным железом, выжигая кожу.", "level": "Severe", "trigger": "пытки"},
]

# 3. НЕНОРМАТИВНАЯ ЛЕКСИКА (добавляем 50 примеров)
profanity_scenarios = [
    # Mild (20 примеров)
    {"text": "Черт, я опоздал на встречу!", "level": "Mild", "trigger": "жаргон 'черт'"},
    {"text": "Блин, это просто ужасно.", "level": "Mild", "trigger": "эвфемизм 'блин'"},
    {"text": "Ему было просто до фени все.", "level": "Mild", "trigger": "сленг 'до фени'"},
    {"text": "Она послала его куда подальше.", "level": "Mild", "trigger": "идиоматическое выражение"},

    # Moderate (15 примеров)
    {"text": "Пошел ты к черту со своими советами!", "level": "Moderate", "trigger": "грубое выражение"},
    {"text": "Это полная хрень, не слушай его.", "level": "Moderate", "trigger": "грубая лексика 'хрень'"},
    {"text": "Он назвал ее дурой при всех.", "level": "Moderate", "trigger": "оскорбительное высказывание"},
    {"text": "Иди к чертовой матери!", "level": "Moderate", "trigger": "грубое ругательство"},

    # Severe (15 примеров)
    {"text": "*****, опять эти проблемы!", "level": "Severe", "trigger": "обсценная лексика"},
    {"text": "***** твою мать, что ты сделал!", "level": "Severe", "trigger": "матерное оскорбление"},
    {"text": "Пошел на *** со своими деньгами.", "level": "Severe", "trigger": "нецензурное выражение"},
    {"text": "*****, это просто невыносимо.", "level": "Severe", "trigger": "мат в предложении"},
]

# 4. АЛКОГОЛЬ/НАРКОТИКИ (добавляем 60 примеров)
alcohol_scenarios = [
    # Mild (20 примеров)
    {"text": "Они выпили по бокалу вина за ужином.", "level": "Mild", "trigger": "упоминание алкоголя"},
    {"text": "На столе стояла бутылка коньяка.", "level": "Mild", "trigger": "нейтральное упоминание"},
    {"text": "Он заказал пиво к ужину.", "level": "Mild", "trigger": "употребление пива"},

    # Moderate (20 примеров)
    {"text": "Он напился в стельку на вечеринке.", "level": "Moderate", "trigger": "сильное опьянение"},
    {"text": "Она приняла таблетку от головной боли.", "level": "Moderate", "trigger": "употребление лекарств"},
    {"text": "Они курили сигареты на балконе.", "level": "Moderate", "trigger": "курение"},

    # Severe (20 примеров)
    {"text": "Он вводил себе героин в вену дрожащими руками.", "level": "Severe", "trigger": "инъекция наркотиков"},
    {"text": "Она нюхала кокаин через свернутую купюру.", "level": "Severe", "trigger": "употребление кокаина"},
    {"text": "Они устроили запой, не выходя из квартиры три дня.", "level": "Severe", "trigger": "детализированный запой"},
]

# 5. ПУГАЮЩИЕ СЦЕНЫ (оставляем как есть, но добавляем None примеров)
fear_scenarios = [
    # None примеры (20 примеров)
    {"text": "Они спокойно пили чай на кухне.", "level": "None", "trigger": ""},
    {"text": "Солнце светило ярко, птицы пели.", "level": "None", "trigger": ""},
    {"text": "Она читала книгу в уютном кресле.", "level": "None", "trigger": ""},
]

# 6. НЕЙТРАЛЬНЫЕ примеры (None для всех категорий) - 100 примеров
neutral_scenarios = [
    {"text": "Они обсуждали планы на выходные.", "level": "None", "trigger": ""},
    {"text": "Погода сегодня прекрасная для прогулки.", "level": "None", "trigger": ""},
    {"text": "Она готовила ужин, напевая любимую песню.", "level": "None", "trigger": ""},
    {"text": "Он работал за компьютером, сосредоточенно печатая.", "level": "None", "trigger": ""},
    {"text": "Дети играли в парке, смеясь и бегая.", "level": "None", "trigger": ""},
]

def create_synthetic_example(scenario, category):
    """Создает полный пример в формате JSON"""
    return {
        "chunk_id": f"synthetic_{category}_{random.randint(1000,9999)}",
        "chunk_text": scenario["text"],
        "assessments": {
            "эротический_контент": {"level": scenario["level"] if category == "sex" else "None", "trigger": scenario["trigger"] if category == "sex" else ""},
            "ненормативная_лексика": {"level": scenario["level"] if category == "profanity" else "None", "trigger": scenario["trigger"] if category == "profanity" else ""},
            "насилие": {"level": scenario["level"] if category == "violence" else "None", "trigger": scenario["trigger"] if category == "violence" else ""},
            "пугающие_сцены": {"level": scenario["level"] if category == "fear" else "None", "trigger": scenario["trigger"] if category == "fear" else ""},
            "алкоголь_наркотики": {"level": scenario["level"] if category == "alcohol" else "None", "trigger": scenario["trigger"] if category == "alcohol" else ""},
            "прочее": {"level": "None", "trigger": ""}
        },
        "final_rating": "None" if scenario["level"] == "None" else "18+"
    }

# Генерируем синтетические данные
synthetic_data = []

# Добавляем примеры для каждой категории
for scenario in erotic_scenarios:
    synthetic_data.append(create_synthetic_example(scenario, "sex"))

for scenario in violence_scenarios:
    synthetic_data.append(create_synthetic_example(scenario, "violence"))

for scenario in profanity_scenarios:
    synthetic_data.append(create_synthetic_example(scenario, "profanity"))

for scenario in alcohol_scenarios:
    synthetic_data.append(create_synthetic_example(scenario, "alcohol"))

for scenario in fear_scenarios:
    synthetic_data.append(create_synthetic_example(scenario, "fear"))

for scenario in neutral_scenarios:
    synthetic_data.append(create_synthetic_example(scenario, "neutral"))

# Смешиваем с оригинальными данными
extended_data = synthetic_data + train_data
random.shuffle(extended_data)

In [4]:
import torch
from unsloth import FastLanguageModel

print("PyTorch:", torch.__version__)
print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
PyTorch: 2.8.0+cu126
CUDA: True
GPU: Tesla T4


In [5]:
from datasets import Dataset
import torch
from unsloth import FastLanguageModel

synthetic_data = extended_data

train_data = synthetic_data
test_data = test_data
print(f"Train examples: {len(train_data)}")
print(f"Test examples: {len(test_data)}")

def check_category_distribution(data, name):
    categories = {
        'эротический_контент': [],
        'насилие': [],
        'ненормативная_лексика': [],
        'алкоголь_наркотики': [],
        'пугающие_сцены': [],
        'прочее': []
    }

    for example in data:
        rates = example["assessments"]
        for category in categories:
            # Исправлено: обращаемся к level внутри категории
            if rates[category]["level"] != "None":
                categories[category].append(rates[category]["level"])

    print(f"\n{name} распределение:")
    for category, values in categories.items():
        print(f"  {category}: {len(values)} не-нейтральных")

check_category_distribution(train_data, "TRAIN")
check_category_distribution(test_data, "TEST")

Train examples: 280
Test examples: 150

TRAIN распределение:
  эротический_контент: 23 не-нейтральных
  насилие: 57 не-нейтральных
  ненормативная_лексика: 33 не-нейтральных
  алкоголь_наркотики: 13 не-нейтральных
  пугающие_сцены: 90 не-нейтральных
  прочее: 28 не-нейтральных

TEST распределение:
  эротический_контент: 4 не-нейтральных
  насилие: 37 не-нейтральных
  ненормативная_лексика: 15 не-нейтральных
  алкоголь_наркотики: 1 не-нейтральных
  пугающие_сцены: 64 не-нейтральных
  прочее: 17 не-нейтральных


In [34]:
SYSTEM_PROMPT = """Ты эксперт по возрастному рейтингу контента. Оцени текст и ОБЯЗАТЕЛЬНО кратко поясни причину оценки.

КРИТЕРИИ ОЦЕНКИ:
ЭРОТИЧЕСКИЙ КОНТЕНТ:
- None: нет контента
- Mild: романтические намёки, поцелуи (поцеловались, обнялись)
- Moderate: откровенные сцены без детализации (раздевание, интимные намёки)
- Severe: детальные сексуальные описания (порнографические сцены) → 18+

НАСИЛИЕ:
- None: нет насилия
- Mild: конфликт "за кадром", упоминания без деталей (поссорились, угрожали)
- Moderate: видимые последствия, краткие описания боли/крови (ударил, кровотечение)
- Severe: натуралистичные описания жестокости (пытки, издевательства) → 18+

НЕНОРМАТИВНАЯ ЛЕКСИКА:
- None: нормативная лексика
- Mild: жаргон, эвфемизмы (черт, блин)
- Moderate: грубая лексика на границе нормы (единичные сильные выражения)
- Severe: обсценная лексика, мат (явный или замаскированный) → 18+

АЛКОГОЛЬ_НАРКОТИКИ:
- None: нет упоминаний
- Mild: краткие нейтральные упоминания (выпил вино)
- Moderate: употребление без подробностей (напился, принял наркотики)
- Severe: детализированное употребление, пропаганда → 18+

ПУГАЮЩИЕ СЦЕНЫ:
- None: нет пугающего контента
- Mild: напряжённая атмосфера (страшилки, напряжение)
- Moderate: страх с угрозой вреда (крики, угрозы)
- Severe: детальный ужас, психологическое насилие → 16+/18+

СРАЗУ SEVERE: призывы к опасным действиям, суицид, дискриминация

Формат ответа СТРОГО:
Sex: уровень // причина
Violence: уровень // причина
Profanity: уровень // причина
Alcohol: уровень // причина
Fear: уровень // причина

Примеры:
Текст: "Они нежно поцеловались"
Sex: Mild // поцелуй
Violence: None // нет насилия
Profanity: None // нормативная лексика
Alcohol: None // нет упоминаний
Fear: None // нет страха

Текст: "Он выхватил пистолет и выстрелил"
Sex: None // нет сексуального контента
Violence: Severe // применение оружия
Profanity: None // нормативная лексика
Alcohol: None // нет упоминаний
Fear: Severe // угроза оружием"""

In [6]:
# создание промптов
def create_training_prompt(example):
    system_prompt = """Ты эксперт по возрастному рейтингу контента. Оцени текст и ОБЯЗАТЕЛЬНО кратко поясни причину оценки.

КРИТЕРИИ ОЦЕНКИ:
ЭРОТИЧЕСКИЙ КОНТЕНТ:
- None: нет контента
- Mild: романтические намёки, поцелуи (поцеловались, обнялись)
- Moderate: откровенные сцены без детализации (раздевание, интимные намёки)
- Severe: детальные сексуальные описания (порнографические сцены) → 18+

НАСИЛИЕ:
- None: нет насилия
- Mild: конфликт "за кадром", упоминания без деталей (поссорились, угрожали)
- Moderate: видимые последствия, краткие описания боли/крови (ударил, кровотечение)
- Severe: натуралистичные описания жестокости (пытки, издевательства) → 18+

НЕНОРМАТИВНАЯ ЛЕКСИКА:
- None: нормативная лексика
- Mild: жаргон, эвфемизмы (черт, блин)
- Moderate: грубая лексика на границе нормы (единичные сильные выражения)
- Severe: обсценная лексика, мат (явный или замаскированный) → 18+

АЛКОГОЛЬ_НАРКОТИКИ:
- None: нет упоминаний
- Mild: краткие нейтральные упоминания (выпил вино)
- Moderate: употребление без подробностей (напился, принял наркотики)
- Severe: детализированное употребление, пропаганда → 18+

ПУГАЮЩИЕ СЦЕНЫ:
- None: нет пугающего контента
- Mild: напряжённая атмосфера (страшилки, напряжение)
- Moderate: страх с угрозой вреда (крики, угрозы)
- Severe: детальный ужас, психологическое насилие → 16+/18+

СРАЗУ SEVERE: призывы к опасным действиям, суицид, дискриминация

Формат ответа СТРОГО:
Sex: уровень // причина
Violence: уровень // причина
Profanity: уровень // причина
Alcohol: уровень // причина
Fear: уровень // причина

Примеры:
Текст: "Они нежно поцеловались"
Sex: Mild // поцелуй
Violence: None // нет насилия
Profanity: None // нормативная лексика
Alcohol: None // нет упоминаний
Fear: None // нет страха

Текст: "Он выхватил пистолет и выстрелил"
Sex: None // нет сексуального контента
Violence: Severe // применение оружия
Profanity: None // нормативная лексика
Alcohol: None // нет упоминаний
Fear: Severe // угроза оружием"""

    # Используем данные из вашего JSON
    user_text = example["chunk_text"]
    assessments = example["assessments"]

    # Создаем ground_truth на основе структуры вашего JSON
    ground_truth = f"""Sex: {assessments['эротический_контент']['level']} // {assessments['эротический_контент']['trigger']}
Violence: {assessments['насилие']['level']} // {assessments['насилие']['trigger']}
Profanity: {assessments['ненормативная_лексика']['level']} // {assessments['ненормативная_лексика']['trigger']}
Alcohol: {assessments['алкоголь_наркотики']['level']} // {assessments['алкоголь_наркотики']['trigger']}
Fear: {assessments['пугающие_сцены']['level']} // {assessments['пугающие_сцены']['trigger']}"""

    prompt = f"""<|im_start|>system
{system_prompt}<|im_end|>
<|im_start|>user
{user_text}<|im_end|>
<|im_start|>assistant
{ground_truth}<|im_end|>"""

    return {"text": prompt}

# Создаем датасеты
train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)

train_dataset = train_dataset.map(create_training_prompt)
test_dataset = test_dataset.map(create_training_prompt)

print("Датасеты созданы!")
print(f"Train промптов: {len(train_dataset)}")
print(f"Test промптов: {len(test_dataset)}")

# # Посмотрим на первый пример для проверки
# print("\nПервый пример промпта:")
# print(train_dataset[0]["text"])

Map:   0%|          | 0/280 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Датасеты созданы!
Train промптов: 280
Test промптов: 150


In [7]:
# Ячейка 3 - загрузка модели и QLoRA
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-3B-unsloth-bnb-4bit",
    max_seq_length=1024,
    dtype=None,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.11.2: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [8]:
# # Посчитай длину твоих текстов
# text_lengths = [len(tokenizer.encode(ex["text"])) for ex in train_dataset]

# import numpy as np
# print(f"Средняя длина: {np.mean(text_lengths):.0f} токенов")
# print(f"Максимальная длина: {np.max(text_lengths)} токенов")
# print(f"95-й перцентиль: {np.percentile(text_lengths, 95):.0f} токенов")

# # Идеальная формула:
# optimal_length = int(np.percentile(text_lengths, 95))  # покрывает 95% примеров
# max_seq_length = min(2048, optimal_length)  # но не больше твоих 6GB VRAM

# print(f"\nРекомендуемая max_seq_length: {max_seq_length}")

In [9]:
# Оптимальная QLoRA конфигурация
model = FastLanguageModel.get_peft_model(
    model,
    r=32,                    # Достаточная capacity
    lora_alpha=64,           # Хорошее масштабирование
    lora_dropout=0.1,       # регуляризация
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    use_gradient_checkpointing=True,  # Экономия памяти
    random_state=42,
)

print("Модель загружена и настроена!")

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.11.2 patched 36 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Модель загружена и настроена!


In [20]:
# Ячейка 4 - обучение с валидацией каждые 5 шагов и сбалансированными весами
from transformers import TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer
import torch
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# ВЫЧИСЛЯЕМ СБАЛАНСИРОВАННЫЕ ВЕСА КЛАССОВ
def compute_class_weights(dataset):
    """Вычисляет сбалансированные веса для основной категории"""

    level_mapping = {"None": 0, "Mild": 1, "Moderate": 2, "High": 3, "Severe": 3}
    levels = []

    for example in dataset:
        assessment = example.get("assessments", {})
        # Используем основную категорию "насилие", можно изменить на нужную
        cat_assessment = assessment.get("насилие", {})
        level = cat_assessment.get("level", "None")
        levels.append(level_mapping.get(level, 0))

    weights = compute_class_weight(
        'balanced',
        classes=np.unique(levels),
        y=levels
    )

    print(f"🎯 Веса классов: {weights}")
    print(f"🎯 Распределение уровней: {np.unique(levels, return_counts=True)}")
    return torch.tensor(weights, dtype=torch.float32)

# Вычисляем веса
class_weights = compute_class_weights(train_dataset)

# Кастомный колбек для мониторинга
from transformers import TrainerCallback

class LossMonitorCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        test_loss = metrics.get("eval_loss", None)
        if test_loss is not None:
            print(f"🚨 TEST_LOSS на шаге {state.global_step}: {test_loss:.4f}")
            if test_loss <= 0.1:
                print("🎯 ЦЕЛЬ ДОСТИГНУТА! Test_loss <= 0.1")

    def on_log(self, args, state, control, logs, **kwargs):
        if "loss" in logs:
            print(f"📊 Train_loss на шаге {state.global_step}: {logs['loss']:.4f}")

# ПРОСТОЙ И БЕЗОПАСНЫЙ СПОСОБ: Добавляем веса через кастомную функцию потерь
def weighted_loss(model_output, labels, class_weights):
    """Кастомная функция потерь с весами классов"""
    loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model_output.logits.device))
    return loss_fct(model_output.logits.view(-1, model_output.logits.size(-1)), labels.view(-1))

    # ВАРИАНТ 2: Используем стандартный тренер и применяем веса через колбек
    class WeightedLossCallback(TrainerCallback):
        def on_step_begin(self, args, state, control, **kwargs):
            # Можно добавить логику применения весов здесь
            pass

    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        dataset_text_field="chunk_text",
        max_seq_length=1024,
        data_collator=data_collator,
        args=training_args,
        callbacks=[LossMonitorCallback(), WeightedLossCallback()],
    )

print("Начинаем обучение со сбалансированными весами...")
print(f"Используемые веса: {class_weights}")
print("=" * 50)

# Параметры обучения
training_args = TrainingArguments(
    # Основные параметры
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,

    # Расписание обучения
    max_steps=50,
    learning_rate=3e-4,
    warmup_steps=20,

    # Валидация и сохранение
    eval_steps=5,
    eval_strategy="steps",
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # Оптимизация
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),

    # Логирование
    logging_steps=5,
    report_to="none",
    output_dir="content_classifier_best",

    # Для воспроизводимости
    seed=42,
    data_seed=42,
)

# ВАРИАНТ 1: Используем кастомный тренер (может вызвать ошибки)
try:
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        dataset_text_field="chunk_text",
        max_seq_length=1024,
        data_collator=data_collator,
        args=training_args,
        callbacks=[LossMonitorCallback()],
    )
    print("✅ Используется кастомный тренер со взвешенными потерями")

except Exception as e:
    print(f"⚠️ Ошибка с кастомным тренером: {e}")
    print("🔄 Используем стандартный тренер")

🎯 Веса классов: [0.31390135 4.11764706 3.68421053 3.33333333]
🎯 Распределение уровней: (array([0, 1, 2, 3]), array([223,  17,  19,  21]))
Начинаем обучение со сбалансированными весами...
Используемые веса: tensor([0.3139, 4.1176, 3.6842, 3.3333])


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/280 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/150 [00:00<?, ? examples/s]

✅ Используется кастомный тренер со взвешенными потерями


In [ ]:
# # Ячейка 4 - обучение с валидацией каждые 5 шагов
# from transformers import TrainingArguments, DataCollatorForLanguageModeling
# from trl import SFTTrainer

# # Data collator
# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer,
#     mlm=False,
# )

# # Кастомный колбек для мониторинга
# from transformers import TrainerCallback

# class LossMonitorCallback(TrainerCallback):
#     def on_evaluate(self, args, state, control, metrics, **kwargs):
#         test_loss = metrics.get("eval_loss", None)
#         if test_loss is not None:
#             print(f"🚨 TEST_LOSS на шаге {state.global_step}: {test_loss:.4f}")
#             if test_loss <= 0.1:
#                 print("🎯 ЦЕЛЬ ДОСТИГНУТА! Test_loss <= 0.1")

#     def on_log(self, args, state, control, logs, **kwargs):
#         if "loss" in logs:
#             print(f"📊 Train_loss на шаге {state.global_step}: {logs['loss']:.4f}")

# # Параметры обучения
# training_args = TrainingArguments(
#     # Основные параметры
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     gradient_accumulation_steps=8,

#     # Расписание обучения
#     max_steps=50,           # Максимум шагов
#     learning_rate=3e-4,      # Оптимальный LR
#     warmup_steps=20,

#     # Валидация и сохранение
#     eval_steps=5,            # ✅ ВАЛИДАЦИЯ КАЖДЫЕ 5 ШАГОВ!
#     eval_strategy="steps",
#     save_steps=50,
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss",
#     greater_is_better=False,

#     # Оптимизация
#     lr_scheduler_type="cosine",
#     weight_decay=0.01,
#     fp16=not torch.cuda.is_bf16_supported(),
#     bf16=torch.cuda.is_bf16_supported(),

#     # Логирование
#     logging_steps=5,
#     report_to="none",
#     output_dir="content_classifier_best",

#     # Для воспроизводимости
#     seed=42,
#     data_seed=42,
# )

# # Создаем тренер
# trainer = SFTTrainer(
#     model=model,
#     tokenizer=tokenizer,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,  # Используем test как validation
#     dataset_text_field="text",
#     max_seq_length=1024,
#     data_collator=data_collator,
#     args=training_args,
#     callbacks=[LossMonitorCallback()],  # Добавляем мониторинг
# )

# print("Начинаем обучение с мониторингом test_loss...")
# print("Цель: test_loss <= 0.2")
# print("=" * 50)

In [21]:
# Ячейка 5 - запуск обучения
try:
    trainer.train()

    # Проверяем финальный результат
    final_metrics = trainer.evaluate()
    final_test_loss = final_metrics["eval_loss"]

    print("=" * 50)
    print("ОБУЧЕНИЕ ЗАВЕРШЕНО!")
    print(f"Финальный test_loss: {final_test_loss:.4f}")

    if final_test_loss <= 0.2:
        print("🎉 ПОЗДРАВЛЯЮ! Цель достигнута!")
    else:
        print("⚠ Цель не достигнута, нужно больше шагов или tuning")

except KeyboardInterrupt:
    print("Обучение прервано пользователем")

except Exception as e:
    print(f"Ошибка во время обучения: {e}")

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 280 | Num Epochs = 6 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 59,867,136 of 3,145,805,824 (1.90% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
5,1.599000,1.369771
10,1.089500,0.697145
15,0.429300,0.271346
20,0.229900,0.237612
25,0.206600,0.220701
30,0.186100,0.211420
35,0.174400,0.206474
40,0.164600,0.204113
45,0.169700,0.202830
50,0.165600,0.202586


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


📊 Train_loss на шаге 5: 1.5990
🚨 TEST_LOSS на шаге 5: 1.3698
📊 Train_loss на шаге 10: 1.0895
🚨 TEST_LOSS на шаге 10: 0.6971
📊 Train_loss на шаге 15: 0.4293
🚨 TEST_LOSS на шаге 15: 0.2713
📊 Train_loss на шаге 20: 0.2299
🚨 TEST_LOSS на шаге 20: 0.2376
📊 Train_loss на шаге 25: 0.2066
🚨 TEST_LOSS на шаге 25: 0.2207
📊 Train_loss на шаге 30: 0.1861
🚨 TEST_LOSS на шаге 30: 0.2114
📊 Train_loss на шаге 35: 0.1744
🚨 TEST_LOSS на шаге 35: 0.2065
📊 Train_loss на шаге 40: 0.1646
🚨 TEST_LOSS на шаге 40: 0.2041
📊 Train_loss на шаге 45: 0.1697
🚨 TEST_LOSS на шаге 45: 0.2028
📊 Train_loss на шаге 50: 0.1656
🚨 TEST_LOSS на шаге 50: 0.2026


🚨 TEST_LOSS на шаге 50: 0.2170
ОБУЧЕНИЕ ЗАВЕРШЕНО!
Финальный test_loss: 0.2170
⚠ Цель не достигнута, нужно больше шагов или tuning


In [36]:
# # Ячейка 8 - сохранение
model.save_pretrained("content_classifier_lora")
tokenizer.save_pretrained("content_classifier_lora")

print("Модель сохранена в папке 'content_classifier_lora'")

Модель сохранена в папке 'content_classifier_lora'


In [37]:
# Ячейка 7 - тестирование сохраненной модели
from unsloth import FastLanguageModel
import pandas as pd

# Загружаем сохраненную модель
model_path = "content_classifier_lora"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=1024,
    dtype=None,
    load_in_4bit=True,
)

# Функция для классификации (адаптированная под сохраненную модель)
def classify_content(text):
    system_prompt = SYSTEM_PROMPT

    prompt = f"""<|im_start|>system
{system_prompt}<|im_end|>
<|im_start|>user
{text}<|im_end|>
<|im_start|>assistant
"""

    inputs = tokenizer([prompt], return_tensors="pt", padding=True).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        temperature=0.1,
        do_sample=False,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Извлекаем только ответ ассистента
    if "<|im_start|>assistant" in full_response:
        assistant_response = full_response.split("<|im_start|>assistant")[-1].strip()
    else:
        assistant_response = full_response

    # Очищаем от возможных лишних частей
    assistant_response = assistant_response.split("<|im_end|>")[0].strip()

    return assistant_response

# Переводим модель в режим инференса
FastLanguageModel.for_inference(model)

# Тестируем
print("=== ТЕСТИРОВАНИЕ СОХРАНЕННОЙ МОДЕЛИ (50 шагов) ===")


test_texts=[x["chunk_text"] for x in test_data]

for i, text in enumerate(test_texts, 1):
    print(f"\n--- Пример {i} ---")
    print(f"Текст: {text}")
    result = classify_content(text)
    print(f"Оценка:\n{result}")
    print("-" * 50)

print("✅ Тестирование завершено!")

Выходные данные были обрезаны до нескольких последних строк (5000).
- Severe: детальные сексуальные описания (порнографические сцены) → 18+

НАСИЛИЕ:
- None: нет насилия
- Mild: конфликт "за кадром", упоминания без деталей (поссорились, угрожали)
- Moderate: видимые последствия, краткие описания боли/крови (ударил, кровотечение)
- Severe: натуралистичные описания жестокости (пытки, издевательства) → 18+

НЕНОРМАТИВНАЯ ЛЕКСИКА:
- None: нормативная лексика
- Mild: жаргон, эвфемизмы (черт, блин)
- Moderate: грубая лексика на границе нормы (единичные сильные выражения)
- Severe: обсценная лексика, мат (явный или замаскированный) → 18+

АЛКОГОЛЬ_НАРКОТИКИ:
- None: нет упоминаний
- Mild: краткие нейтральные упоминания (выпил вино)
- Moderate: употребление без подробностей (напился, принял наркотики)
- Severe: детализированное употребление, пропаганда → 18+

ПУГАЮЩИЕ СЦЕНЫ:
- None: нет пугающего контента
- Mild: напряжённая атмосфера (страшилки, напряжение)
- Moderate: страх с угрозой вреда (

In [ ]:
import numpy as np
from sklearn.metrics import recall_score, classification_report
from collections import defaultdict

def calculate_recall_on_test(model, tokenizer, test_dataset):
    """
    Расчет recall для многозадачной классификации на тестовом датасете
    """

    # Маппинг уровней в числовые значения
    level_mapping = {"None": 0, "Mild": 1, "Moderate": 2, "High": 3, "Severe": 3}

    # Категории для оценки
    categories = ["эротический_контент", "ненормативная_лексика", "насилие",
                  "пугающие_сцены", "алкоголь_наркотики", "прочее"]

    # Храним истинные и предсказанные метки
    true_labels = {category: [] for category in categories}
    pred_labels = {category: [] for category in categories}

    model.eval()

    print("🔍 Начинаем оценку на тестовом датасете...")

    for i, example in enumerate(test_dataset):
        chunk_text = example["chunk_text"]

        # Генерируем предсказание
        inputs = tokenizer(chunk_text, return_tensors="pt", truncation=True, max_length=1024)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=100,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=False
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Парсим истинные метки из датасета
        assessments = example["assessments"]
        for category in categories:
            true_level = assessments.get(category, {}).get("level", "None")
            true_labels[category].append(level_mapping.get(true_level, 0))

        # Парсим предсказанные метки из ответа модели
        # Это зависит от формата вывода вашей модели
        pred_levels = parse_model_response(response, categories)

        for category in categories:
            pred_level = pred_levels.get(category, "None")
            pred_labels[category].append(level_mapping.get(pred_level, 0))

        if (i + 1) % 50 == 0:
            print(f"Обработано {i + 1} примеров...")

    # Вычисляем recall для каждой категории
    print("\n" + "="*60)
    print("📊 РЕЗУЛЬТАТЫ RECALL ПО КАТЕГОРИЯМ:")
    print("="*60)

    category_recalls = {}

    for category in categories:
        y_true = true_labels[category]
        y_pred = pred_labels[category]

        # Recall для каждого класса отдельно
        recalls_per_class = recall_score(y_true, y_pred, average=None, zero_division=0)
        # Macro-average recall (учитывает дисбаланс классов)
        macro_recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
        # Weighted recall (учитывает размеры классов)
        weighted_recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)

        category_recalls[category] = {
            'macro_recall': macro_recall,
            'weighted_recall': weighted_recall,
            'per_class_recall': recalls_per_class
        }

        print(f"\n🎯 {category}:")
        print(f"   Macro Recall: {macro_recall:.4f}")
        print(f"   Weighted Recall: {weighted_recall:.4f}")
        print(f"   По классам: {recalls_per_class}")

        # Детальный отчет
        print(f"\n   Детальный отчет:")
        print(classification_report(y_true, y_pred,
                                  target_names=['None', 'Mild', 'Moderate', 'High'],
                                  zero_division=0))

    # Общая статистика
    print("\n" + "="*60)
    print("📈 ОБЩАЯ СТАТИСТИКА:")
    print("="*60)

    avg_macro_recall = np.mean([cat['macro_recall'] for cat in category_recalls.values()])
    avg_weighted_recall = np.mean([cat['weighted_recall'] for cat in category_recalls.values()])

    print(f"Средний Macro Recall: {avg_macro_recall:.4f}")
    print(f"Средний Weighted Recall: {avg_weighted_recall:.4f}")

    # Лучшие и худшие категории
    best_category = max(category_recalls.items(), key=lambda x: x[1]['macro_recall'])
    worst_category = min(category_recalls.items(), key=lambda x: x[1]['macro_recall'])

    print(f"\n🏆 Лучшая категория: {best_category[0]} (Recall: {best_category[1]['macro_recall']:.4f})")
    print(f"📉 Худшая категория: {worst_category[0]} (Recall: {worst_category[1]['macro_recall']:.4f})")

    return category_recalls

def parse_model_response(response, categories):
    """
    Парсит ответ модели и извлекает предсказания для каждой категории
    Нужно адаптировать под ваш формат вывода модели!
    """
    pred_levels = {}

    # Пример парсинга - адаптируйте под ваш формат!
    for category in categories:
        # Ищем паттерн "категория: уровень" в ответе
        import re
        pattern = f"{category}:\s*(\w+)"
        match = re.search(pattern, response)

        if match:
            pred_levels[category] = match.group(1)
        else:
            # Если не нашли - ставим "None" по умолчанию
            pred_levels[category] = "None"

    return pred_levels

# Альтернативная упрощенная версия для быстрого тестирования
def quick_recall_test(model, tokenizer, test_dataset, num_samples=100):
    """
    Быстрый тест recall на ограниченной выборке
    """
    print("🚀 Быстрый тест recall...")

    # Берем подвыборку
    import random
    if len(test_dataset) > num_samples:
        test_subset = random.sample(list(test_dataset), num_samples)
    else:
        test_subset = test_dataset

    return calculate_recall_on_test(model, tokenizer, test_subset)

# Использование:
recalls = calculate_recall_on_test(model=model, tokenizer=tokenizer, test_dataset=test_dataset)
recalls = quick_recall_test(model, tokenizer, test_dataset, num_samples=50)

In [ ]:
# # Ячейка 8 - сохранение
# model.save_pretrained("content_classifier_lora")
# tokenizer.save_pretrained("content_classifier_lora")

# print("Модель сохранена в папке 'content_classifier_lora'")

In [38]:
!zip -r my_model.zip /content/content_classifier_lora

  adding: content/content_classifier_lora/ (stored 0%)
  adding: content/content_classifier_lora/merges.txt (deflated 57%)
  adding: content/content_classifier_lora/README.md (deflated 65%)
  adding: content/content_classifier_lora/adapter_config.json (deflated 56%)
  adding: content/content_classifier_lora/vocab.json (deflated 61%)
  adding: content/content_classifier_lora/special_tokens_map.json (deflated 68%)
  adding: content/content_classifier_lora/adapter_model.safetensors (deflated 9%)
  adding: content/content_classifier_lora/added_tokens.json (deflated 67%)
  adding: content/content_classifier_lora/tokenizer_config.json (deflated 89%)
  adding: content/content_classifier_lora/tokenizer.json (deflated 81%)


In [1]:
# Архивируем папку с моделью
!zip -r model_08_11.zip /content/content_classifier_lora/

# Скачиваем
from google.colab import files
files.download("model_08_11.zip")

	zip warning: name not matched: /content/content_classifier_lora/

zip error: Nothing to do! (try: zip -r model_08_11.zip . -i /content/content_classifier_lora/)


FileNotFoundError: Cannot find file: model_08_11.zip

In [2]:
model

NameError: name 'model' is not defined